## Optimization model 

In [1]:
from model import shortestPathModel

In [2]:
# model for shortest path
grid = (5,5)
sp_model = shortestPathModel(grid)

Academic license - for non-commercial use only - expires 2021-04-13
Using license file C:\Users\Apocrypse\gurobi.lic


In [3]:
# solve
sp_model.setObj([1 for i in range(40)])
sol, obj = sp_model.solve()
print('Obj: {}'.format(obj))
for i, e in enumerate(sp_model.arcs):
    if sol[i] > 1e-3:
        print(e)

Obj: 8.0
(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 9)
(9, 14)
(14, 19)
(19, 24)


## Data Loader

In [4]:
from data import shortestpath, dataset
from torch.utils.data import DataLoader

In [5]:
# generate data (features and costs)
n = 100
p = 5
x, c = shortestpath.genData(n, p, grid)

In [6]:
# get data set
sp_dataset = dataset.optDataset(sp_model, x, c)

Optimizing...


100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 194.49it/s]


In [7]:
# get data loader
sp_loader = DataLoader(sp_dataset, batch_size=32, shuffle=True)

In [8]:
for i, data in enumerate(sp_loader):
    x, c, w, z = data
    break

In [9]:
x.shape

torch.Size([32, 5])

In [10]:
c.shape

torch.Size([32, 40])

In [11]:
w.shape

torch.Size([32, 40])

In [12]:
z.shape

torch.Size([32, 1])

## Loss Function

In [13]:
from loss import SPOPlusLoss

In [14]:
# init SPO+ loss
criterion = SPOPlusLoss()

## Linear Regression 

In [15]:
import torch
from torch import nn

In [16]:
# build linear model
class LinearRegression(nn.Module):
    
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(p, (grid[0] - 1) * grid[1] + (grid[1] - 1) * grid[0])
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [17]:
# cuda
if torch.cuda.is_available():
    lr = LinearRegression().cuda()
else:
    lr = LinearRegression()

In [18]:
# set optimizer
optimizer = torch.optim.SGD(lr.parameters(), lr=1e-2)

In [19]:
# train model
num_epochs = 100
for epoch in range(num_epochs):
    print('Start epoch {}:'.format(epoch))
    # load data
    for i, data in enumerate(sp_loader):
        x, c, w, z = data
        # cuda
        x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
        # forward pass
        cp = lr(x)
        loss = criterion.apply(sp_model, cp, c, w, z).mean()
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 1 == 0:
            print('    [{}/{}], loss:{:.6f}'.format(i, len(sp_loader)-1, loss.item()))

Start epoch 0:
    [0/3], loss:8.195473
    [1/3], loss:9.586136
    [2/3], loss:8.752203
    [3/3], loss:8.193995
Start epoch 1:
    [0/3], loss:8.305334
    [1/3], loss:7.893786
    [2/3], loss:9.615836
    [3/3], loss:7.972273
Start epoch 2:
    [0/3], loss:9.638073
    [1/3], loss:7.893947
    [2/3], loss:8.013460
    [3/3], loss:4.906394
Start epoch 3:
    [0/3], loss:7.897882
    [1/3], loss:7.311571
    [2/3], loss:9.516828
    [3/3], loss:8.155360
Start epoch 4:
    [0/3], loss:7.925223
    [1/3], loss:7.603507
    [2/3], loss:8.150341
    [3/3], loss:12.943519
Start epoch 5:
    [0/3], loss:7.310102
    [1/3], loss:9.246223
    [2/3], loss:7.278535
    [3/3], loss:7.661371
Start epoch 6:
    [0/3], loss:7.613308
    [1/3], loss:7.815911
    [2/3], loss:8.119655
    [3/3], loss:6.871319
Start epoch 7:
    [0/3], loss:7.081855
    [1/3], loss:7.690882
    [2/3], loss:8.487394
    [3/3], loss:5.749568
Start epoch 8:
    [0/3], loss:7.187903
    [1/3], loss:8.216187
    [2/3], los

    [0/3], loss:3.753890
    [1/3], loss:3.792802
    [2/3], loss:3.987155
    [3/3], loss:2.271375
Start epoch 72:
    [0/3], loss:3.587864
    [1/3], loss:4.406548
    [2/3], loss:3.159031
    [3/3], loss:5.021093
Start epoch 73:
    [0/3], loss:3.187597
    [1/3], loss:3.926374
    [2/3], loss:3.963956
    [3/3], loss:5.228552
Start epoch 74:
    [0/3], loss:4.622186
    [1/3], loss:2.948512
    [2/3], loss:3.727470
    [3/3], loss:3.155368
Start epoch 75:
    [0/3], loss:4.321337
    [1/3], loss:3.414782
    [2/3], loss:3.495333
    [3/3], loss:2.687109
Start epoch 76:
    [0/3], loss:3.102440
    [1/3], loss:4.228324
    [2/3], loss:3.849012
    [3/3], loss:2.511009
Start epoch 77:
    [0/3], loss:3.802533
    [1/3], loss:3.747111
    [2/3], loss:3.505922
    [3/3], loss:2.725001
Start epoch 78:
    [0/3], loss:4.287692
    [1/3], loss:3.322599
    [2/3], loss:3.165029
    [3/3], loss:4.372651
Start epoch 79:
    [0/3], loss:3.831040
    [1/3], loss:4.049599
    [2/3], loss:3.0481